### MMLU

In [25]:
from datasets import load_dataset

ds = load_dataset("cais/mmlu", "all")
## Stratified sample on 'subject' field
MAX_PER_SUBJ = 50

test_ds = ds['test'].to_pandas()
test_sample = test_ds.groupby('subject', group_keys=False).apply(lambda x: x.sample(min(len(x), MAX_PER_SUBJ), random_state=42)).reset_index(drop=True)


test_sample.to_json('../Self-Distillation/data/mmlu/eval_data.json', orient='records', lines=True)

/tmp/ipykernel_1398915/487230862.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_sample = test_ds.groupby('subject', group_keys=False).apply(lambda x: x.sample(min(len(x), MAX_PER_SUBJ), random_state=42)).reset_index(drop=True)


In [ ]:
test_ds.subject

0        abstract_algebra
1        abstract_algebra
2        abstract_algebra
3        abstract_algebra
4        abstract_algebra
               ...       
14037     world_religions
14038     world_religions
14039     world_religions
14040     world_religions
14041     world_religions
Name: subject, Length: 14042, dtype: object

### Tools

In [100]:
import pandas as pd
import os

train_df = pd.read_json('../Self-Distillation/data/tooluse_data/train_data.json')
print("N Rows: ", len(train_df))
print("Columns: ", train_df.columns)

subsample_df = train_df.sample(1000, random_state=42)
os.makedirs("../Self-Distillation/data/tooluse_train/", exist_ok=True)

# subsample_df.to_json('../Self-Distillation/data/tooluse_train/eval_data.json', orient='records')

N Rows:  4046
Columns:  Index(['prompt', 'name', 'description', 'nl_documentation', 'instruction',
       'golden_answer', 'golden_response'],
      dtype='object')


In [101]:
train_df

,prompt,name,description,nl_documentation,instruction,golden_answer,golden_response
0,Your task is to answer the user's question usi...,Httpbin,A Simple HTTP Request & Response Service,sendHttpRequest: Send an HTTP request with the...,"I'm troubleshooting some requests, so can you ...","[{'Action': 'sendHttpRequest', 'Action_Input':...",[ I need to use the sendHttpRequest tool to se...
1,Your task is to answer the user's question usi...,Httpbin,A Simple HTTP Request & Response Service,sendHttpRequest: Send an HTTP request with the...,Could you send a GET request to the following ...,"[{'Action': 'getClientRequestData', 'Action_In...",[ I need to use the getClientRequestData tool ...
2,Your task is to answer the user's question usi...,Httpbin,A Simple HTTP Request & Response Service,sendHttpRequest: Send an HTTP request with the...,I have a feeling that some proxy headers aren'...,"[{'Action': 'sendHttpRequest', 'Action_Input':...",[ I need to send an HTTP request with the spec...
3,Your task is to answer the user's question usi...,Httpbin,A Simple HTTP Request & Response Service,sendHttpRequest: Send an HTTP request with the...,I'd like to see how my application responds to...,"[{'Action': 'simulateStatusCode', 'Action_Inpu...",[ I need to use the simulateStatusCode tool to...
4,Your task is to answer the user's question usi...,Httpbin,A Simple HTTP Request & Response Service,sendHttpRequest: Send an HTTP request with the...,"I need to test how my app handles a ""PATCH"" re...","[{'Action': 'sendHttpRequest', 'Action_Input':...",[ I need to use the sendHttpRequest tool to se...
...,...,...,...,...,...,...,...
4041,Your task is to answer the user's question usi...,Bible-api,Free Bible API with multiple languages,getVerse: Retrieve the text of a specific vers...,"Hey, can you search the Bible for the keyword ...","[{'Action': 'search', 'Action_Input': '{""query...",[ I need to use the search tool to find the ke...
4042,Your task is to answer the user's question usi...,Bible-api,Free Bible API with multiple languages,getVerse: Retrieve the text of a specific vers...,I need to find all the available Bible version...,"[{'Action': 'getVersions', 'Action_Input': '{""...",[ I need to use the getVersions tool to find t...
4043,Your task is to answer the user's question usi...,Bible-api,Free Bible API with multiple languages,getVerse: Retrieve the text of a specific vers...,Can you give me the details of all the Bible v...,"[{'Action': 'getVersions', 'Action_Input': '{""...",[ I need to use the getVersions tool to get th...
4044,Your task is to answer the user's question usi...,Bible-api,Free Bible API with multiple languages,getVerse: Retrieve the text of a specific vers...,I want to find a Bible version in Portuguese. ...,"[{'Action': 'getVersions', 'Action_Input': '{""...",[ I need to use the getVersions tool to find t...


In [88]:
subsample_df = pd.read_json('../Self-Distillation/data/tooluse_train/eval_data.json')
subsample_df.prompt[0]

'Your task is to answer the user\'s question using available tools. \nYou have access to the following tools:\nName: Httpbin\nDescription: A Simple HTTP Request & Response Service\nDocumentation:\nsendHttpRequest: Send an HTTP request with the specified method, headers, and data to the Httpbin API for testing purposes.\nParameters: {"method": "Required. string. One of: [GET, POST, PUT, DELETE, HEAD, PATCH]. The HTTP method to use (GET, POST, PUT, DELETE, HEAD, or PATCH).", "url": "Required. string. The endpoint URL to send the request to.", "headers": "Object.  A key-value pair of headers to include in the request.", "data": "Object.  A key-value pair of data to include in the request body."}\nOutput: Successful response.\n - Format: application/json\n - Structure: Object{response: Object{status_code, headers: Object, body}}\ngetClientRequestData: Retrieve the client\'s request data, including headers, form data, uploaded files, and cookies.\nParameters: {"url": "Required. string. The 

In [40]:
import pandas as pd

eval_df = pd.read_json('../Self-Distillation/data/tooluse_data/eval_data.json')
print("N Rows: ", len(eval_df))
print("Columns: ", eval_df.columns)

N Rows:  68
Columns:  Index(['prompt', 'name', 'description', 'nl_documentation', 'instruction',
       'golden_answer'],
      dtype='object')


In [50]:
print(eval_df.prompt[1])

Your task is to answer the user's question using available tools. 
You have access to the following tools:
Name: Axolotl
Description: Collection of axolotl pictures and facts
Documentation:
getRandomAxolotlImage: Retrieve a random axolotl image with information on the image source.
Parameters: {}
Output: Successful response.
 - Format: application/json
 - Structure: Object{url, source, description}
searchAxolotlImages: Search for axolotl images based on specific criteria such as color, gender, and size.
Parameters: {"color": "string. One of: [wild, leucistic, albino]. The color of the axolotl (e.g., 'wild', 'leucistic', 'albino', etc.).", "gender": "string. One of: [male, female]. The gender of the axolotl ('male', 'female').", "size": "string. One of: [small, medium, large]. The size of the axolotl ('small', 'medium', 'large').", "page": "integer. The page number for pagination purposes."}
Output: Successful response.
 - Format: application/json
 - Structure: Object{results: Array[Obj

In [72]:
import pandas as pd

sim_df = pd.read_json('../Self-Distillation/data/tooluse_data/eval_simulated.json')
real_df = pd.read_json('../Self-Distillation/data/tooluse_data/eval_real.json')

print("Simulated Data:")
print("N Rows: ", len(sim_df))
print("Columns: ", sim_df.columns)

print("Real Data:")
print("N Rows: ", len(real_df))
print("Columns: ", real_df.columns)

Simulated Data:
N Rows:  10
Columns:  Index(['Name', 'Description', 'Link', 'Category', 'Introduction', 'Functions',
       'Documentation', 'NLDocumentation', 'Function_Description',
       'Function_Projection', 'Instructions', 'Golden_Answers'],
      dtype='object')
Real Data:
N Rows:  11
Columns:  Index(['Name', 'Description', 'Category', 'Authentication', 'external_tools',
       'Documentation', 'NLDocumentation', 'Function_Description',
       'Function_Projection', 'Instructions', 'Golden_Answers'],
      dtype='object')


In [79]:
sim_df.Instructions.apply(len).sum(), real_df.Instructions.apply(len).sum()

(np.int64(100), np.int64(114))

In [90]:
eval_df = pd.read_json("../Self-Distillation/data/tooluse_data/eval_data.json")
sim_df = pd.read_json("../Self-Distillation/data/tooluse_sim/eval_data.json")

In [91]:
eval_df.columns, sim_df.columns

(Index(['prompt', 'name', 'description', 'nl_documentation', 'instruction',
        'golden_answer'],
       dtype='object'),
 Index(['prompt', 'name', 'description', 'nl_documentation', 'instruction',
        'golden_answer'],
       dtype='object'))

### SciKnowEval - Chem

In [17]:
import os 
import pandas as pd
import numpy as np

In [3]:
files_dir = "../Self-Distillation/data/sciknoweval/"

file_paths = [os.path.join(files_dir, fname) for fname in os.listdir(files_dir) ]

In [20]:
df = pd.concat([pd.read_json(fp, lines=True) for fp in file_paths], ignore_index=True)
df.prompt = df.prompt.apply(lambda x: x['default'] if isinstance(x, dict) else x)

In [21]:
df

,prompt,question,answer,type,domain,details,answerKey,choices
0,You are an expert chemist. Given a chemical eq...,Here is a unbalanced chemical equation:\nNa2Cr...,Na2CrO4(aq) + 2LiBr(aq) = Li2CrO4(aq) + 2NaBr(s),filling,Chemistry,"{'level': 'L3', 'task': 'balancing_chemical_eq...",,"{'text': [], 'label': []}"
1,You are an expert chemist. Given a chemical eq...,Here is a unbalanced chemical equation:\nBa(NO...,Ba(NO3)2 + H2SO4 = BaSO4 + 2HNO3,filling,Chemistry,"{'level': 'L3', 'task': 'balancing_chemical_eq...",,"{'text': [], 'label': []}"
2,You are an expert chemist. Given a chemical eq...,Here is a unbalanced chemical equation:\nNH4 +...,NH4 + NO2 = 2H2O + 2N,filling,Chemistry,"{'level': 'L3', 'task': 'balancing_chemical_eq...",,"{'text': [], 'label': []}"
3,You are an expert chemist. Given a chemical eq...,Here is a unbalanced chemical equation:\nClO3{...,ClO3{-} + 2Ag{+} + 2OH{-} = ClO4{-} + 2Ag + H2O,filling,Chemistry,"{'level': 'L3', 'task': 'balancing_chemical_eq...",,"{'text': [], 'label': []}"
4,You are an expert chemist. Given a chemical eq...,Here is a unbalanced chemical equation:\nMnO4 ...,MnO4 + 2CO = MnO2 + 2CO2,filling,Chemistry,"{'level': 'L3', 'task': 'balancing_chemical_eq...",,"{'text': [], 'label': []}"
...,...,...,...,...,...,...,...,...
2395,"Given a question and four options, please sele...",Which of the following molecules is the only c...,,mcq-4-choices,Chemistry,"{'level': 'L3', 'task': 'retrosynthesis', 'sub...",D,{'text': ['COC(=O)c1cc(C)c([N+](=O)[O-])c(C)c1...
2396,"Given a question and four options, please sele...",Which of the following molecules is the only c...,,mcq-4-choices,Chemistry,"{'level': 'L3', 'task': 'retrosynthesis', 'sub...",D,{'text': ['CC(C)[C@@H](C(=O)N[C@H]1CCc2ccn3c2C...
2397,"Given a question and four options, please sele...",Which of the following molecules is the only c...,,mcq-4-choices,Chemistry,"{'level': 'L3', 'task': 'retrosynthesis', 'sub...",C,{'text': ['CCn1cc(CO)c(=O)c2cc(OCc3ccc(OC)cc3)...
2398,"Given a question and four options, please sele...",Which of the following molecules is the only c...,,mcq-4-choices,Chemistry,"{'level': 'L3', 'task': 'retrosynthesis', 'sub...",C,{'text': ['CC(C)(C)OC(=O)N1CC[C@H]1COc1cccnc1'...


In [36]:
x=df[df.type=='filling'].sample(1)

print(x.prompt.values)
print(x.question.values)
print(x.answer.values)

['You are an expert chemist. Given a chemical equation, please balance the equation without any explanation and maintain the order of reactants and products as given.']
['Here is a unbalanced chemical equation:\nNaI + (NH4)2CO3 = Na2CO3 + NH4I\nThe balanced chemical equation is:']
['2NaI + (NH4)2CO3 = Na2CO3 + 2NH4I']


In [45]:
x=df[df.type=='mcq-4-choices'].sample(1)

print(x.prompt.values)
print(x.question.values)
print(x.choices.values[0])
print(x.answerKey.values)


['Given a question and four options, please select the right answer. Your answer should be "A", "B", "C" or "D". Please directly give the answer without any explanation.']
["What is the molar weight (g/mol) of the molecule with the canonical SMILES representation 'C1CCC2C(C1)CC3N2C(=O)C4CC5=CC=CC=C5CN4C(=O)C(CCNC(=O)C(NC3=O)CCCN=C(N)N)NC(=O)C(CC6=CC=CS6)NC(=O)CNC(=O)C7CC(CN7C(=O)C8CCCN8C(=O)C(CCCN=C(N)N)NC(=O)C(CCCCN)N)O'?"]
{'text': ['1347.600', '1362.600', '1271.500', '1261.400'], 'label': ['A', 'B', 'C', 'D']}
['C']


In [ ]:
for i, row in df.iterrows():
    if row.type=='mcq-4-choices':
        prompt = row.prompt + "\n" + row.question + "\nChoices: " + ", ".join(row.choices) + "\nAnswer:"
        answer = row.answerKey
        break


In [54]:
print(prompt)

Given a question and four options, please select the right answer. Your answer should be "A", "B", "C" or "D". Please directly give the answer without any explanation.
Question: What is the molar weight (g/mol) of the molecule with the the IUPAC name '(2S)-2-[[(2S)-2-[[(2S)-2-[[(2S)-5-amino-2-[[2-[[(2S)-2-[[(2S)-2-[[(2S)-2-[[(2S)-2-[[(2S)-2-[[(2S)-2-[[(2S)-2-[[(2S)-1-[(2S)-2-[[(2S,3R)-2-amino-3-hydroxybutanoyl]amino]-3-hydroxypropanoyl]pyrrolidine-2-carbonyl]amino]-5-(diaminomethylideneamino)pentanoyl]amino]-5-(diaminomethylideneamino)pentanoyl]amino]-4-methylpentanoyl]amino]-3-methylbutanoyl]amino]-4-carboxybutanoyl]amino]-4-methylpentanoyl]amino]propanoyl]amino]acetyl]amino]-5-oxopentanoyl]amino]-3-hydroxypropanoyl]amino]-4-methylpentanoyl]amino]-4-methylpentanoic acid'?
Choices: text, label
Answer:


In [55]:
"\n".join()

{'text': ['1639.900', '1674.800', '1683.000', '1583.700'],
 'label': ['A', 'B', 'C', 'D']}

### MMLU Caps

In [27]:
import pandas as pd
# train test split import
from sklearn.model_selection import train_test_split

df1 = pd.read_json("/home/mila/b/baldelld/scratch/arena-capstone/Self-Distillation/data/mmlu-caps/mmlu-caps-self-generation-eval.json")
df2 = pd.read_json("/home/mila/b/baldelld/scratch/arena-capstone/Self-Distillation/data/mmlu-caps/mmlu-caps-self-generation.json")

pd.concat([df1, df2], ignore_index=True).to_json("/home/mila/b/baldelld/scratch/arena-capstone/Self-Distillation/data/mmlu-caps/all_data.json", orient='records')

In [32]:
import pandas as pd
# train test split import
from sklearn.model_selection import train_test_split

df = pd.read_json('../Self-Distillation/data/mmlu-caps/all_data.json')
train_df, eval_df = train_test_split(df, test_size=0.1, random_state=42)

# Rename for compatibility with training pipeline
train_df = train_df.rename(columns={'golden_answer': 'golden_response'})
eval_df = eval_df.rename(columns={'golden_answer': 'golden_answer'})  # eval already correct

train_df.to_json('../Self-Distillation/data/mmlu-caps/train_data.json', orient='records')
eval_df.to_json('../Self-Distillation/data/mmlu-caps/eval_data.json', orient='records')

In [31]:
x = eval_df.sample(1)

print(x.prompt.values[0])
print(x.golden_answer.values[0])

A 27-year-old man is brought to the emergency department by his sister because of increasing confusion for 10 hours. He is unable to answer questions. His sister states that he recently saw a psychiatrist for the first time because of hearing voices; he was prescribed a medication, but she is not sure what it is. She says that he has a history of excessive drinking, and she thinks that he has also experimented with illicit drugs. He appears acutely ill. His temperature is 39.1°C (102.3°F), pulse is 124/min, and blood pressure is 160/102 mm Hg. Examination shows profuse diaphoresis and muscle rigidity. His neck is supple. The abdomen is soft and nontender. Mental status examination shows psychomotor agitation alternating with lethargy. His leukocyte count is 15,600/mm3 , and serum creatine kinase activity is 943 U/L. Which of the following is the most likely explanation for this patient's symptoms?
BASED ON THE CLINICAL PRESENTATION, THE MOST LIKELY EXPLANATION FOR THIS PATIENT'S SYMPTO

In [33]:
len(train_df), len(eval_df)

(7531, 837)

In [23]:
2916 / 1632

1.786764705882353

In [24]:
1632 / 2916

0.5596707818930041

In [36]:
data_size=7531

## Distill
n_gups = 4
per_dev_batch_size = 1
num_gen = 8
gradient_accumulation_steps = 2

effective_batch_size = n_gups * per_dev_batch_size * gradient_accumulation_steps * num_gen

n_epochs=1
n_samples=data_size*n_epochs

training_steps = (n_samples // effective_batch_size)

print(f"Distill BS: {effective_batch_size} - N Samples: {n_samples} - Steps: {training_steps}")

## SFT
n_gpus=1
per_device_batch_size=16
gradient_accumulation_steps = 2

effective_batch_size = n_gpus * per_device_batch_size * gradient_accumulation_steps
n_epochs=8
n_samples=data_size*n_epochs

training_steps = (n_samples // effective_batch_size)

print(f"SFT BS: {effective_batch_size} - N Samples: {n_samples} - Steps: {training_steps}")

Distill BS: 64 - N Samples: 7531 - Steps: 117
SFT BS: 32 - N Samples: 60248 - Steps: 1882
